In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
import sys, os, re, csv, codecs, numpy as np, pandas as pd
from fastai import *        # Quick accesss to most common functionality
from fastai.text import *   # Quick accesss to NLP functionality
import html

Taken from here:  
https://gist.github.com/bearpelican/48cd4c505aea7c94e7c1e6e5e24bfac0

# This notebook expects files to be located in train,test,valid folders

# Wikitext 103
This notebook is for training the language model on most of Wikipedia.  
The idea is to create a generalized language model before we fine tune it on a specialized task

In [3]:
Path.home()

PosixPath('/home/ubuntu')

In [4]:
PATH=Path.home()/'data/wikitext-103-fmt'

In [5]:
# PATH=Path.home()/'data/wikitext-2-raw'

### Test: Flip pad with unk

In [6]:
# defaults.text_spec_tok = [PAD,UNK,BOS,FLD,TK_MAJ,TK_UP,TK_REP,TK_WREP]

In [7]:
defaults.text_post_rules = [defaults.text_post_rules[0]]

In [8]:
# download_url('https://s3.amazonaws.com/research.metamind.io/wikitext/wikitext-103-raw-v1.zip', PATH)

In [9]:
def read_file(fn:PathOrStr, enc='utf-8'):
    "Read the text in `fn`."
#     with open(fn,'r', encoding = enc) as f: return ''.join(f.read().splitlines())
    tokens = []
    with open(fn,'r', encoding = enc) as f: 
        for line in f.read().splitlines():
            l = line.strip()
            if len(l) == 0: continue
            tokens.append(l)#.split())
    return np.array(tokens)

In [10]:
class OpenLargeFileProcessor(PreProcessor):
    "`PreProcessor` that opens the filenames and read the texts."
#     def process(self, ds:Collection):        ds.items = [self.process_one(item) for item in ds.items]
#     def process_one(self,item): return open_3(item) if isinstance(item, Path) else item
    def process(self, ds:Collection): ds.items = read_file(ds.items[0])

In [11]:
p = [OpenLargeFileProcessor(), 
     TokenizeProcessor(chunksize=1000), 
     NumericalizeProcessor(vocab=None)]

In [12]:
data = (TextList.from_folder(PATH, recurse=True, extensions=['.raw'], processor=p)
        .split_by_folder()
        .label_for_lm()
        .databunch(bs=100))

BrokenProcessPool: A process in the process pool was terminated abruptly while the future was running or pending.

In [ ]:
data.save()

### Wikitext-2

In [8]:
PATH=Path('data/wikitext-2-raw')

In [7]:
# download_url('https://s3.amazonaws.com/research.metamind.io/wikitext/wikitext-2-raw-v1.zip', PATH)

In [9]:
p = [OpenLargeFileProcessor(), 
     TokenizeProcessor(chunksize=1000), 
     NumericalizeProcessor(vocab=None)]

In [10]:
data = (TextList.from_folder(PATH, recurse=True, extensions=['.raw'], processor=p)
        .split_by_folder()
        .label_for_lm()
        .databunch(bs=100))

/home/ubuntu/fastai/fastai/data_block.py:398: UserWarning: Your training set is empty. Is this is by design, pass `ignore_empty=True` to remove this warning.
  warn("Your training set is empty. Is this is by design, pass `ignore_empty=True` to remove this warning.")
/home/ubuntu/fastai/fastai/data_block.py:401: UserWarning: Your validation set is empty. Is this is by design, use `no_split()` 
                 or pass `ignore_empty=True` when labelling to remove this warning.
  or pass `ignore_empty=True` when labelling to remove this warning.""")


IndexError: index 0 is out of bounds for axis 0 with size 0

In [15]:
data.save()

In [16]:
ob = data.one_batch(); [x.shape for x in ob]

[torch.Size([100, 95]), torch.Size([100, 95])]

In [17]:
data.show_batch()

idx,text
0,"xxbos xxmaj polka xxmaj party ! received mixed to negative reviews from critics . xxmaj allmusic reviewer xxmaj eugene xxmaj xxunk gave the album three stars and wrote that "" just about anyone could feel let down by this album . "" xxmaj xxunk was largely critical of the parody choices , noting that many of the original versions would be forgotten in "" fifteen years "" . xxmaj christopher"
1,the influence continued through the 20th century . xxmaj gerald xxmaj kelly recalled xxmaj bertin when painting his restless and confined series of portraits of xxmaj ralph xxmaj vaughan xxmaj williams in 1952 – 61 . xxmaj in 1975 xxmaj xxunk xxmaj xxunk produced a series of nine black and white photographs on board based on xxmaj ingres ' portraits of xxmaj bertin and xxmaj xxunk xxmaj caroline xxmaj rivière
2,") , a life - saving organisation equipped with technologically - advanced land , sea , air and space rescue craft ; these are headed by a fleet of five vehicles named the xxmaj thunderbirds and launched from xxup ir 's secret base in the xxmaj pacific xxmaj ocean . xxmaj the main characters are ex - astronaut xxmaj jeff xxmaj tracy , the founder of xxup ir , and"
3,"xxmaj open , losing to xxmaj xxunk in the quarterfinals and xxunk his xxmaj no . 1 ranking , having been just one week away from xxunk xxmaj pete xxmaj sampras 's record of 286 weeks as world xxmaj no . 1 . xxmaj in a huge upset at xxmaj wimbledon , xxmaj federer lost in the quarterfinals to xxmaj tomáš xxmaj berdych and fell to xxmaj no . 3"
4,"1 , with xxmaj don xxmaj bradman scoring xxunk runs at a batting average of xxunk , an aggregate record that still stands . xxmaj by the time of the next xxmaj ashes series of 1932 – 33 , xxmaj bradman 's average xxunk around 100 , approximately twice that of all other world - class batsmen . xxmaj england feared that without xxunk to drastic tactics , they might"


### Loading data

In [4]:
data = TextLMDataBunch.load(PATH)

### Note:
Before, preparing this dataset would cause the transformer to nan out. This is because we were transforming `<unk>` tokens to xxunk which had an index of 0. However, the transformer had a hardcoded embedding padding index of 0. Thus, when you attempted to get embedding of index 0, it returned an embedding of all nan's.